In [11]:
import tensorflow as tf
import mnist_data
import tensorflowvisu
tf.set_random_seed(0)

In [2]:
mnist = mnist_data.read_data_sets("data") #data is the folder name

Loading data/train-images-idx3-ubyte.mnist
Loading data/train-labels-idx1-ubyte.mnist
Loading data/t10k-images-idx3-ubyte.mnist
Loading data/t10k-labels-idx1-ubyte.mnist


In [3]:
print 'Training Examples:', mnist.train.images.shape[0]
print 'Testing  Examples:', mnist.test.images.shape[0]

#mnist.train.labels[0]

Training Examples: 60000
Testing  Examples: 10000


In [4]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y_ = tf.placeholder(tf.float32, [None, 10]) # correct answers 


W = tf.Variable(tf.zeros([784, 10])) # weights W[784, 10] t.random_normal instead of tf.zeroes?
b = tf.Variable(tf.zeros([10]))

# flatten the images into a single line of pixels
# -1 in the shape definition means "the only possible dimension that will preserve the number of elements"
XX = tf.reshape(X, [-1, 784])

In [5]:
# The model
Y = tf.nn.softmax(tf.matmul(XX, W) + b)

In [6]:
# loss function: cross-entropy = - sum( Y_i * log(Yi) )
#                           Y: the computed output vector
#                           Y_: the desired output vector

# cross-entropy
# log takes the log of each element, * multiplies the tensors element by element
# reduce_mean will add all the components in the tensor
# so here we end up with the total cross-entropy for all images in the batch
cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y)) * 1000.0  # normalized for batches of 100 images,
                                                          # *10 because  "mean" included an unwanted division by 10

In [7]:
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# training, learning rate = 0.005
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(cross_entropy)

In [13]:
# matplotlib visualisation
allweights = tf.reshape(W, [-1])
allbiases = tf.reshape(b, [-1])
I = tensorflowvisu.tf_format_mnist_images(X, Y, Y_)  # assembles 10x10 images by default
It = tensorflowvisu.tf_format_mnist_images(X, Y, Y_, 1000, lines=25)  # 1000 images on 25 lines
datavis = tensorflowvisu.MnistDataVis()

# init
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [14]:
def training_step(i, update_test_data, update_train_data):

    # training on batches of 100 images with 100 labels
    batch_X, batch_Y = mnist.train.next_batch(100)

    # compute training values for visualisation
    if update_train_data:
        a, c, im, w, b = sess.run([accuracy, cross_entropy, I, allweights, allbiases], feed_dict={X: batch_X, Y_: batch_Y})
        datavis.append_training_curves_data(i, a, c)
        datavis.append_data_histograms(i, w, b)
        datavis.update_image1(im)
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c))

    # compute test values for visualisation
    if update_test_data:
        a, c, im = sess.run([accuracy, cross_entropy, It], feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
        datavis.append_test_curves_data(i, a, c)
        datavis.update_image2(im)
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))

    # the backpropagation training step
    sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y})

In [15]:
datavis.animate(training_step, iterations=2000+1, train_data_update_freq=10, test_data_update_freq=50, more_tests_at_start=True)

# to save the animation as a movie, add save_movie=True as an argument to datavis.animate
# to disable the visualisation use the following line instead of the datavis.animate line
# for i in range(2000+1): training_step(i, i % 50 == 0, i % 10 == 0)

print("max test accuracy: " + str(datavis.get_max_test_accuracy()))

0: accuracy:0.13 loss: 230.259
0: ********* epoch 1 ********* test accuracy:0.098 test loss: 230.26


/home/diego/anaconda2/lib/python2.7/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


10: accuracy:0.6 loss: 115.933
10: ********* epoch 1 ********* test accuracy:0.6787 test loss: 104.211
20: accuracy:0.83 loss: 61.2917
20: ********* epoch 1 ********* test accuracy:0.8488 test loss: 59.6142
30: accuracy:0.81 loss: 63.7258
30: ********* epoch 1 ********* test accuracy:0.8639 test loss: 52.2593
40: accuracy:0.88 loss: 45.9763
40: ********* epoch 1 ********* test accuracy:0.8704 test loss: 48.1346
50: accuracy:0.88 loss: 46.4928
50: ********* epoch 1 ********* test accuracy:0.8728 test loss: 46.7755
60: accuracy:0.95 loss: 28.2206
70: accuracy:0.86 loss: 53.9742
80: accuracy:0.89 loss: 31.6543
90: accuracy:0.92 loss: 31.7568
100: accuracy:0.91 loss: 38.8968
100: ********* epoch 1 ********* test accuracy:0.8812 test loss: 41.3588
110: accuracy:0.93 loss: 26.7649
120: accuracy:0.88 loss: 36.2421
130: accuracy:0.76 loss: 69.8536
140: accuracy:0.85 loss: 46.7728
150: accuracy:0.95 loss: 25.0937
150: ********* epoch 1 ********* test accuracy:0.8992 test loss: 35.9475
160: accu